## Using Python to Perform Extract-Transform-Load (ETL Processing)
Modern Data Warehousing and Analytics solutions frequently use languages like Python or Scala to extract data from numerous sources, including relational database management systems, NoSQL database systems, real-time streaming endpoints and Data Lakes.  These languages can then be used to perform many types of transformation before then loading the data into a variety of destinations including file systems and data warehouses. This data can then be consumed by data scientists or business analysts.

In this lab you will recreate the **Northwind_DW** dimensional database from Lab 2; however, you'll take an entirely different approach. Instead of extracting, transforming and loading the date entirely on the database system entirely using SQL data definition language (DDL) and data manipulation language (DML) statements, here you will learn to interact with the RDBMS from a remote client running Python. You will learn to fetch data into Pandas DataFrames, perform all the necessary transformations in-memory on the client, and then push the newly transformed DataFrame back to the RDBMS using a Pandas function that will create the table and fill it with data with a single operation.

### Prerequisites:
#### Import the Necessary Libraries

In [1]:
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine

#### Declare & Assign Connection Variables for the MySQL Server & Databases with which You'll be Working 

In [2]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "Passw0rd123"

src_dbname = "northwind"
dst_dbname = "northwind_dw2"

#### Define Functions for Getting Data From and Setting Data Into Databases

In [3]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

#### Create the New Data Warehouse database, and to Use it, Switch the Connection Context.
Clearly, you won't get very far without having a database to work with. Here we demonstrate how we can *drop* a database if it already exists, and then *create* the new **northwind_dw2** database and *use* it as the target of all subsequent operations.

In [4]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

### 1.0. Create & Populate the Dimension Tables
#### 1.1. Extract Data from the Source Database Tables

In [5]:
sql_customers = "SELECT * FROM northwind.customers;"
df_customers = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customers)
df_customers.head(2)

,id,company,last_name,first_name,email_address,job_title,business_phone,home_phone,mobile_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page,notes,attachments
0,1,Company A,Bedecs,Anna,None,Owner,(123)555-0100,None,None,(123)555-0101,123 1st Street,Seattle,WA,99999,USA,None,None,b''
1,2,Company B,Gratacos Solsona,Antonio,None,Owner,(123)555-0100,None,None,(123)555-0101,123 2nd Street,Boston,MA,99999,USA,None,None,b''


In [6]:
sql_employees = "SELECT * FROM northwind.employees;"
df_employees = get_dataframe(user_id, pwd, host_name, src_dbname, sql_employees)
df_employees.head(2)

,id,company,last_name,first_name,email_address,job_title,business_phone,home_phone,mobile_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page,notes,attachments
0,1,Northwind Traders,Freehafer,Nancy,nancy@northwindtraders.com,Sales Representative,(123)555-0100,(123)555-0102,None,(123)555-0103,123 1st Avenue,Seattle,WA,99999,USA,#http://northwindtraders.com#,None,b''
1,2,Northwind Traders,Cencini,Andrew,andrew@northwindtraders.com,"Vice President, Sales",(123)555-0100,(123)555-0102,None,(123)555-0103,123 2nd Avenue,Bellevue,WA,99999,USA,http://northwindtraders.com#http://northwindtr...,"Joined the company as a sales representative, ...",b''


In [7]:
sql_products = "SELECT * FROM northwind.products;"
df_products = get_dataframe(user_id, pwd, host_name, src_dbname, sql_products)
df_products.head(2)

,supplier_ids,id,product_code,product_name,description,standard_cost,list_price,reorder_level,target_level,quantity_per_unit,discontinued,minimum_reorder_quantity,category,attachments
0,4,1,NWTB-1,Northwind Traders Chai,None,13.5,18.0,10,40,10 boxes x 20 bags,0,10.0,Beverages,b''
1,10,3,NWTCO-3,Northwind Traders Syrup,None,7.5,10.0,25,100,12 - 550 ml bottles,0,25.0,Condiments,b''


In [8]:
sql_shippers = "SELECT * FROM northwind.shippers;"
df_shippers = get_dataframe(user_id, pwd, host_name, src_dbname, sql_shippers)
df_shippers.head(2)

,id,company,last_name,first_name,email_address,job_title,business_phone,home_phone,mobile_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page,notes,attachments
0,1,Shipping Company A,None,None,None,None,None,None,None,None,123 Any Street,Memphis,TN,99999,USA,None,None,b''
1,2,Shipping Company B,None,None,None,None,None,None,None,None,123 Any Street,Memphis,TN,99999,USA,None,None,b''


#### 1.2. Perform Any Necessary Transformations
Pandas DataFrames enable extensive data modification capabilities. Here we will start by simply dropping features (columns) that we don't believe provide any real value to our analytics solution. Examples include columns having a high percentage of NULL values, columns having large amounts of free-text, and columns having binary large object (BLOB) data such as images or other documents. Then, we will rename the primary key column (id) to conform with data warehouse design standards.

In [9]:
drop_cols = ['email_address','home_phone','mobile_phone','web_page','notes','attachments']
df_customers.drop(drop_cols, axis=1, inplace=True)
df_customers.rename(columns={"id":"customer_key"}, inplace=True)

df_customers.head(2)

,customer_key,company,last_name,first_name,job_title,business_phone,fax_number,address,city,state_province,zip_postal_code,country_region
0,1,Company A,Bedecs,Anna,Owner,(123)555-0100,(123)555-0101,123 1st Street,Seattle,WA,99999,USA
1,2,Company B,Gratacos Solsona,Antonio,Owner,(123)555-0100,(123)555-0101,123 2nd Street,Boston,MA,99999,USA


In [10]:
drop_cols = ['mobile_phone','notes','attachments']
df_employees.drop(drop_cols, axis=1, inplace=True)
df_employees.rename(columns={"id":"employee_key"}, inplace=True)

df_employees.head(2)

,employee_key,company,last_name,first_name,email_address,job_title,business_phone,home_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page
0,1,Northwind Traders,Freehafer,Nancy,nancy@northwindtraders.com,Sales Representative,(123)555-0100,(123)555-0102,(123)555-0103,123 1st Avenue,Seattle,WA,99999,USA,#http://northwindtraders.com#
1,2,Northwind Traders,Cencini,Andrew,andrew@northwindtraders.com,"Vice President, Sales",(123)555-0100,(123)555-0102,(123)555-0103,123 2nd Avenue,Bellevue,WA,99999,USA,http://northwindtraders.com#http://northwindtr...


In [12]:
drop_cols = ['supplier_ids','description','attachments']
df_products.drop(drop_cols, axis=1, inplace=True)
df_products.rename(columns={"id":"product_key"}, inplace=True)

df_products.head(2)

KeyError: "['supplier_ids', 'description', 'attachments'] not found in axis"

In [13]:
drop_cols = ['last_name','first_name','email_address','job_title','business_phone',
             'home_phone','mobile_phone','fax_number','web_page','notes','attachments']
df_shippers.drop(drop_cols, axis=1, inplace=True)
df_shippers.rename(columns={"id":"shipper_key"}, inplace=True)

df_shippers.head(2)

,shipper_key,company,address,city,state_province,zip_postal_code,country_region
0,1,Shipping Company A,123 Any Street,Memphis,TN,99999,USA
1,2,Shipping Company B,123 Any Street,Memphis,TN,99999,USA


#### 1.4. Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables
Here I demonstrate how we can create an iterable data structure containing the values needed to correctly create and populate the new dimension tables. If you inspect this code listing carefully, you'll notice that it's a **list** containing a **set** *(or vector)* for each dimension table. Each **set** then contains the *table_name* we need to assign to the table, the *pandas DataFrame* we crafted to define & populate the table, and the name we need to assign to the *primary_key* column.  With this *list of sets* defined, we can then call our **set_dataframe( )** function from within a **for *loop*** to create each *dimension* table.

In [14]:
df_employees.head(2)

,employee_key,company,last_name,first_name,email_address,job_title,business_phone,home_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page
0,1,Northwind Traders,Freehafer,Nancy,nancy@northwindtraders.com,Sales Representative,(123)555-0100,(123)555-0102,(123)555-0103,123 1st Avenue,Seattle,WA,99999,USA,#http://northwindtraders.com#
1,2,Northwind Traders,Cencini,Andrew,andrew@northwindtraders.com,"Vice President, Sales",(123)555-0100,(123)555-0102,(123)555-0103,123 2nd Avenue,Bellevue,WA,99999,USA,http://northwindtraders.com#http://northwindtr...


In [16]:
db_operation = "insert"

tables = [('dim_customers', df_customers, 'customer_key'),
          ('dim_employees', df_employees, 'employee_key'),
          #('dim_employees', df_employees, 'id'),
          ('dim_products', df_products, 'product_key'),
          ('dim_shippers', df_shippers, 'shipper_key')]

In [17]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

### 2.0. Create & Populate the Fact Table
Here we will learn two approaches to creating the *fact_orders* fact table. The first approach demonstrates that a carefully crafted SQL SELECT statement can be used to perform this task... *but what fun would that be.* Seriously though, this approach is quick and effect if you already have the query, but what if you didn't have the opportunity to view and work with the data beforehand?  What's more, you may be required to combine data from multiple sources, some of which may not be relational database management systems. Then, a simple SQL query won't do!  You would need to load the data from the various sources (e.g., database tables, CSV or JSON files, NoSQL document collections, API stream data) and then combine them into a single dataframe that you could then use to create a new database table. For this reason we'll see how we can retrieve the data, but we won't bother to use it for creating a new table... we already know how to do that using the **set_dataframe( )** function anyway.

#### 2.1. First, you could simply use the SQL SELECT statement you authored in Lab 2 

In [18]:
sql_query = """
SELECT o.id,
    o.employee_id,
    o.customer_id,
    od.product_id,
    o.shipper_id,
    o.ship_name,
    o.ship_address,
    o.ship_city,
    o.ship_state_province,
    o.ship_zip_postal_code,
    o.ship_country_region,
    od.quantity,
    o.order_date,
    o.shipped_date, 
    od.unit_price,
    od.discount,
    o.shipping_fee,
    o.taxes,
    o.payment_type,
    o.paid_date,
    o.tax_rate,
    os.status_name AS order_status,
    ods.status_name AS order_details_status
FROM northwind.orders AS o
INNER JOIN northwind.orders_status AS os
ON o.status_id = os.id
INNER JOIN northwind.order_details AS od
ON o.id = od.order_id
INNER JOIN northwind.order_details_status AS ods
ON od.status_id = ods.id;
"""

df_fact_orders = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customers)
df_fact_orders

,id,company,last_name,first_name,email_address,job_title,business_phone,home_phone,mobile_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page,notes,attachments
0,1,Company A,Bedecs,Anna,None,Owner,(123)555-0100,None,None,(123)555-0101,123 1st Street,Seattle,WA,99999,USA,None,None,b''
1,2,Company B,Gratacos Solsona,Antonio,None,Owner,(123)555-0100,None,None,(123)555-0101,123 2nd Street,Boston,MA,99999,USA,None,None,b''
2,3,Company C,Axen,Thomas,None,Purchasing Representative,(123)555-0100,None,None,(123)555-0101,123 3rd Street,Los Angelas,CA,99999,USA,None,None,b''
3,4,Company D,Lee,Christina,None,Purchasing Manager,(123)555-0100,None,None,(123)555-0101,123 4th Street,New York,NY,99999,USA,None,None,b''
4,5,Company E,O’Donnell,Martin,None,Owner,(123)555-0100,None,None,(123)555-0101,123 5th Street,Minneapolis,MN,99999,USA,None,None,b''
5,6,Company F,Pérez-Olaeta,Francisco,None,Purchasing Manager,(123)555-0100,None,None,(123)555-0101,123 6th Street,Milwaukee,WI,99999,USA,None,None,b''
6,7,Company G,Xie,Ming-Yang,None,Owner,(123)555-0100,None,None,(123)555-0101,123 7th Street,Boise,ID,99999,USA,None,None,b''
7,8,Company H,Andersen,Elizabeth,None,Purchasing Representative,(123)555-0100,None,None,(123)555-0101,123 8th Street,Portland,OR,99999,USA,None,None,b''
8,9,Company I,Mortensen,Sven,None,Purchasing Manager,(123)555-0100,None,None,(123)555-0101,123 9th Street,Salt Lake City,UT,99999,USA,None,None,b''
9,10,Company J,Wacker,Roland,None,Purchasing Manager,(123)555-0100,None,None,(123)555-0101,123 10th Street,Chicago,IL,99999,USA,None,None,b''


#### 2.2. Instead, Implement the solution using Pandas DataFrames to Craft the table
This is where we get to the point of this lab.  We'll query the source **northwind** database to fill a *dataframe* for each of the source tables we need to create our *fact_orders* fact table; orders, orders_status, order_details and order_details_status. Then, we'll learn how to *join* those *dataframes* using the **merge( )** method of the Pandas DataFrame.  We'll make any additional changes that we expect to see reflected in the *fact* table in our new MySQL database, and then, we'll push the *dataframe* back to the MySQL server to create and populate the new *fact* table.

##### 2.2.1. Get all the data from each of the four tables involved

In [50]:
sql_orders = "SELECT * FROM northwind.orders;"
df_orders = get_dataframe(user_id, pwd, host_name, src_dbname, sql_orders)
df_orders.rename(columns={"id":"order_id"}, inplace=True)
df_orders.head(5)

,order_id,employee_id,customer_id,order_date,shipped_date,shipper_id,ship_name,ship_address,ship_city,ship_state_province,ship_zip_postal_code,ship_country_region,shipping_fee,taxes,payment_type,paid_date,notes,tax_rate,tax_status_id,status_id
0,30,9,27,2006-01-15,2006-01-22,2.0,Karen Toh,789 27th Street,Las Vegas,NV,99999,USA,200.0,0.0,Check,2006-01-15,None,0.0,None,3
1,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,NY,99999,USA,5.0,0.0,Credit Card,2006-01-20,None,0.0,None,3
2,32,4,12,2006-01-22,2006-01-22,2.0,John Edwards,123 12th Street,Las Vegas,NV,99999,USA,5.0,0.0,Credit Card,2006-01-22,None,0.0,None,3
3,33,6,8,2006-01-30,2006-01-31,3.0,Elizabeth Andersen,123 8th Street,Portland,OR,99999,USA,50.0,0.0,Credit Card,2006-01-30,None,0.0,None,3
4,34,9,4,2006-02-06,2006-02-07,3.0,Christina Lee,123 4th Street,New York,NY,99999,USA,4.0,0.0,Check,2006-02-06,None,0.0,None,3


In [51]:
sql_orders_status = "SELECT * FROM northwind.orders_status;"
df_orders_status = get_dataframe(user_id, pwd, host_name, src_dbname, sql_orders_status)
df_orders_status.rename(columns={"id": "status_id"}, inplace=True)
df_orders_status.head(5)

,status_id,status_name
0,0,New
1,1,Invoiced
2,2,Shipped
3,3,Closed


In [44]:
sql_order_details = "SELECT * FROM northwind.order_details;"
df_order_details = get_dataframe(user_id, pwd, host_name, src_dbname, sql_order_details)
df_order_details.head(2)

,id,order_id,product_id,quantity,unit_price,discount,status_id,date_allocated,purchase_order_id,inventory_id
0,27,30,34,100.0,14.0,0.0,2,None,96.0,83.0
1,28,30,80,30.0,3.5,0.0,2,None,NaN,63.0


In [45]:
sql_order_details_status = "SELECT * FROM northwind.order_details_status;"
df_order_details_status = get_dataframe(user_id, pwd, host_name, src_dbname,sql_order_details_status)
df_order_details_status.rename(columns={"id":"status_id"}, inplace=True)
df_order_details_status.head(2)

,status_id,status_name
0,0,None
1,1,Allocated


##### 2.2.2. Get the order_status column.
Here we use the dataframe's **merge( )** method to **inner join** the *orders* and the *orders_status* dataframes **on** the *status_id* column. We then use the dataframe's **rename( )** method to rename the *status_name* column to *order_status*, and use the dataframe's **drop( )** method to remove the *status_id* column.

In [52]:
df_orders =pd.merge(df_orders, df_orders_status, on = 'status_id', how = 'inner')
df_orders.rename(columns={"status_name":"order_status"}, inplace=True)
df_orders.drop(['status_id'], axis=1, inplace=True)

In [53]:
df_orders.head(2)

,order_id,employee_id,customer_id,order_date,shipped_date,shipper_id,ship_name,ship_address,ship_city,ship_state_province,ship_zip_postal_code,ship_country_region,shipping_fee,taxes,payment_type,paid_date,notes,tax_rate,tax_status_id,order_status
0,30,9,27,2006-01-15,2006-01-22,2.0,Karen Toh,789 27th Street,Las Vegas,NV,99999,USA,200.0,0.0,Check,2006-01-15,None,0.0,None,Closed
1,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,NY,99999,USA,5.0,0.0,Credit Card,2006-01-20,None,0.0,None,Closed


##### 2.2.3. Get the order_details_status
Here we repeat the sequence of operations we used in the previous step to **inner join** the *order_details* and *order_details_status* dataframes for the sake of including the *order_details_status* column in place of the *status_id* column.

In [54]:
df_order_details = pd.merge(df_order_details,df_order_details_status, how = 'inner', on = 'status_id')
df_order_details.rename(columns={"status_name":"order_details_status"}, inplace=True)
df_order_details.drop(['status_id'], axis=1, inplace=True)

In [55]:
df_order_details.head(5)

,id,order_id,product_id,quantity,unit_price,discount,date_allocated,purchase_order_id,inventory_id,order_details_status
0,27,30,34,100.0,14.0,0.0,None,96.0,83.0,Invoiced
1,28,30,80,30.0,3.5,0.0,None,NaN,63.0,Invoiced
2,29,31,7,10.0,30.0,0.0,None,NaN,64.0,Invoiced
3,30,31,51,10.0,53.0,0.0,None,NaN,65.0,Invoiced
4,31,31,80,10.0,3.5,0.0,None,NaN,66.0,Invoiced


##### 2.2.4. Join the Orders and OrderDetails DataFrames
In this step we can now easily join the *orders* and *order_details* dataframes. Since each **order** (the *left* dataframe) can have many **order details** (the *right* dataframe), we'll need to implement a **right** *outer join* **on** the *order_id* column.

In [56]:
df_fact_orders = pd.merge(df_orders,df_order_details, how = 'right', on = 'order_id')

In [57]:
df_fact_orders.head(5)

,order_id,employee_id,customer_id,order_date,shipped_date,shipper_id,ship_name,ship_address,ship_city,ship_state_province,...,order_status,id,product_id,quantity,unit_price,discount,date_allocated,purchase_order_id,inventory_id,order_details_status
0,30,9,27,2006-01-15,2006-01-22,2.0,Karen Toh,789 27th Street,Las Vegas,NV,...,Closed,27,34,100.0,14.0,0.0,None,96.0,83.0,Invoiced
1,30,9,27,2006-01-15,2006-01-22,2.0,Karen Toh,789 27th Street,Las Vegas,NV,...,Closed,28,80,30.0,3.5,0.0,None,NaN,63.0,Invoiced
2,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,NY,...,Closed,29,7,10.0,30.0,0.0,None,NaN,64.0,Invoiced
3,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,NY,...,Closed,30,51,10.0,53.0,0.0,None,NaN,65.0,Invoiced
4,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,NY,...,Closed,31,80,10.0,3.5,0.0,None,NaN,66.0,Invoiced


##### 2.2.5. Perform any Additional Transformations
In this step we can prepare the DataFrame so that it defines exactly what we want to see created in the database.  Issues may include dropping unwanted columns, reordering the columns, and in our case, creating a new column to serve as the primary key.

In [58]:
drop_columns = ['id', 'notes', 'tax_status_id', 'purchase_order_id', 'inventory_id', 'date_allocated']
df_fact_orders.drop(drop_columns, axis = 1, inplace = True)

df_fact_orders.head(5)

,order_id,employee_id,customer_id,order_date,shipped_date,shipper_id,ship_name,ship_address,ship_city,ship_state_province,...,taxes,payment_type,paid_date,tax_rate,order_status,product_id,quantity,unit_price,discount,order_details_status
0,30,9,27,2006-01-15,2006-01-22,2.0,Karen Toh,789 27th Street,Las Vegas,NV,...,0.0,Check,2006-01-15,0.0,Closed,34,100.0,14.0,0.0,Invoiced
1,30,9,27,2006-01-15,2006-01-22,2.0,Karen Toh,789 27th Street,Las Vegas,NV,...,0.0,Check,2006-01-15,0.0,Closed,80,30.0,3.5,0.0,Invoiced
2,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,NY,...,0.0,Credit Card,2006-01-20,0.0,Closed,7,10.0,30.0,0.0,Invoiced
3,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,NY,...,0.0,Credit Card,2006-01-20,0.0,Closed,51,10.0,53.0,0.0,Invoiced
4,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,NY,...,0.0,Credit Card,2006-01-20,0.0,Closed,80,10.0,3.5,0.0,Invoiced


In [60]:
ordered_cols = ['order_id', 'employee_id', 'customer_id', 'product_id', 'shipper_id', 'ship_name', 'ship_address', 'ship_city',
               'ship_state_province', 'ship_zip_postal_code','quantity', 'order_date', 'shipped_date', 'unit_price',
               'discount', 'shipping_fee', 'taxes', 'payment_type', 'paid_date', 'order_status', 'order_details_status']

df_fact_orders = df_fact_orders[ordered_cols]

In [61]:
df_fact_orders.head(5)

,order_id,employee_id,customer_id,product_id,shipper_id,ship_name,ship_address,ship_city,ship_state_province,ship_zip_postal_code,...,order_date,shipped_date,unit_price,discount,shipping_fee,taxes,payment_type,paid_date,order_status,order_details_status
0,30,9,27,34,2.0,Karen Toh,789 27th Street,Las Vegas,NV,99999,...,2006-01-15,2006-01-22,14.0,0.0,200.0,0.0,Check,2006-01-15,Closed,Invoiced
1,30,9,27,80,2.0,Karen Toh,789 27th Street,Las Vegas,NV,99999,...,2006-01-15,2006-01-22,3.5,0.0,200.0,0.0,Check,2006-01-15,Closed,Invoiced
2,31,3,4,7,1.0,Christina Lee,123 4th Street,New York,NY,99999,...,2006-01-20,2006-01-22,30.0,0.0,5.0,0.0,Credit Card,2006-01-20,Closed,Invoiced
3,31,3,4,51,1.0,Christina Lee,123 4th Street,New York,NY,99999,...,2006-01-20,2006-01-22,53.0,0.0,5.0,0.0,Credit Card,2006-01-20,Closed,Invoiced
4,31,3,4,80,1.0,Christina Lee,123 4th Street,New York,NY,99999,...,2006-01-20,2006-01-22,3.5,0.0,5.0,0.0,Credit Card,2006-01-20,Closed,Invoiced


In [62]:
# primary key:
df_fact_orders.insert(0, "order_key", range(1, df_fact_orders.shape[0]+1))
df_fact_orders.head(2)

,order_key,order_id,employee_id,customer_id,product_id,shipper_id,ship_name,ship_address,ship_city,ship_state_province,...,order_date,shipped_date,unit_price,discount,shipping_fee,taxes,payment_type,paid_date,order_status,order_details_status
0,1,30,9,27,34,2.0,Karen Toh,789 27th Street,Las Vegas,NV,...,2006-01-15,2006-01-22,14.0,0.0,200.0,0.0,Check,2006-01-15,Closed,Invoiced
1,2,30,9,27,80,2.0,Karen Toh,789 27th Street,Las Vegas,NV,...,2006-01-15,2006-01-22,3.5,0.0,200.0,0.0,Check,2006-01-15,Closed,Invoiced


##### 2.2.6. Write the DataFrame Back to the Database

In [63]:
table_name = "fact_orders"
primary_key = "order_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_fact_orders, table_name, primary_key, db_operation)

### 3.0. Demonstrate that the New Data Warehouse Exists and Contains the Correct Data
To demonstrate the viability of your solution, author a SQL SELECT statement that returns:
-	Each Customer’s Last Name
-	The total amount of the order quantity associated with each customer
-	The total amount of the order unit price associated with each customer

In [64]:
sql_test = """
    SELECT customers.`last_name` AS `customer_name`,
        SUM(orders.`quantity`) AS `total_quantity`,
        SUM(orders.`unit_price`) AS `total_unit_price`
    FROM `{0}`.`fact_orders` AS orders
    INNER JOIN `{0}`.`dim_customers` AS customers
    ON orders.customer_id = customers.customer_key
    GROUP BY customers.`last_name`
    ORDER BY total_unit_price DESC;
""".format(dst_dbname)

df_test = get_dataframe(user_id, pwd, host_name, src_dbname, sql_test)

In [65]:
df_test.head()

,customer_name,total_quantity,total_unit_price
0,Lee,277.0,287.45
1,Pérez-Olaeta,427.0,162.50
2,Raghav,405.0,120.05
3,Andersen,260.0,118.70
4,Axen,253.0,100.64
